In [1]:
# Loading of libraries
import numpy as np
import scipy.spatial.distance

In [190]:
# Design variables
AverageRadius            = 1
DeviationRadius          = AverageRadius / 10
ParticleCount            = 20
Viscosity                = 1E3
ViscosityR               = 32 / 3 * Viscosity
RotationalViscoisty      = 1E3
RotationalViscoistyR     = 4 * np.pi * RotationalViscoisty
NeighbourDistance        = 2.7 * AverageRadius
SpringConstant           = 1
Chi                      = 1         # Rotational viscosity    - 1 in natural units
Zeta                     = 1         # Translational visocsity - 1 in natural units
k                        = 1         # Sprint constant         - 1 in natural units

# Simulation variables
TimeStep                 = 0.01
L_align                  = 0.1       # Range from 0.1 to 1
L_noise                   = 0.04      # Range from 0.04 to 0.08
L_F_in                   = 0.3
L_T_in                   = 3
L_self                   = 0.03
T_align                  = L_align * k * Chi / Zeta
T_noise                  = np.sqrt(L_noise / Zeta / TimeStep * 2 * k * Chi**2)
F_in                     = L_F_in * AverageRadius * k
T_in                     = L_T_in * Chi * k / Zeta
F_self                   = L_self * k * AverageRadius


# Initiate the pengiuins
Radius                   = np.random.normal(AverageRadius, DeviationRadius, ParticleCount)
Location                 = (np.vstack((
                             np.tile(np.arange(0,10), ParticleCount / 10) * 10,
                             np.repeat(np.arange(0,ParticleCount / 10), 10) * 10
                           ))).T# + np.random.normal(0,1,(ParticleCount,2))
Velocity                 = np.zeros([ParticleCount, 2])
Orientation              = np.ones(ParticleCount) * np.pi / 2 + np.random.uniform(-pi/4, pi/4, ParticleCount)
Rotation                 = np.zeros(ParticleCount)


In [191]:
## This function calculates the force on the particles:
# ParticleRadius  = Radius of the particle
# ViscosityR      = Scaled viscosity of the system
# Velocity        = Velocity of the particle
# NeighbourRadius = Radius in which Neighbours can be found
def ParticleForce(ParticleRadius, Orientation, NeighbourRadius):
    # Particle force = Self-Propulsion + Boundary + Repulsion
    Heaviside       = piecewise(NeighbourRadius,
                          [NeighbourRadius < pi, NeighbourRadius > np.pi],
                          [0, 1])
    PropulsionForce = F_self * Orientation
    BoundaryForce   = (NeighbourRadius - np.pi) * F_in * Heaviside * Orientation
    RepulsionForce  = SpringConstant * sum(NeighbourOverlap)
    
    return PropulsionForce + BoundaryForce + RepulsionForce

    
def ParticleTorque(ParticleRadius, RotationalViscoistyR, Rotation):
    Heaviside      = piecewise(NeighbourRadius,
                          [NeighbourRadius < pi, NeighbourRadius > np.pi],
                          [0, 1])
    # Particletorque = Self-Torque + Noise + Align
    BoundaryToruqe = T_in * deltaTheta * Heaviside
    NoiseTorque    = T_noise * random(-1,1,1)
    AlignTorque    = T_align * sum(Theta_neighbour_mismatch)
    
    return BoundaryTorque + NoiseTorque + AlignTorque


## This functions creates a list of neighbours for all particles
#  ParticleLocations is the Nx2 array containing the locations of the particles
def NeighbourList(Location):
    Distances = scipy.spatial.distance.pdist(Location)
    Neighbours = np.nonzero(Distances < 2.7);
    return Neighbours
    
def NeighbourRadius(Location, Neighbours):
    Distance = Location[Neightbours[:,0]] - Location[Neightbours[:,0]];
    print(Distance);
    return;

In [192]:
Neightbours = NeighbourList(Location)
Ne

max_steps = np.int(1E3);
for step in range(0,max_steps):
    Velocity = Velocity;

(array([], dtype=int64), array([], dtype=int64))
